In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Read in the CSV files from previous script
record_df = pd.read_csv("resources/record.csv")
ats_df = pd.read_csv("resources/ats.csv")
ou_df = pd.read_csv("resources/ou.csv")

In [3]:
# Create a unique identifier to combine team and year
record_df['UID'] = record_df['Year'].astype(str) + record_df['Team'].astype(str)
ats_df['UID'] = ats_df['Year'].astype(str) + ats_df['Team'].astype(str)
ou_df['UID'] = ou_df['Year'].astype(str) + ou_df['Team'].astype(str)

# Merge the DataFrames into a larger dataset
df1 = pd.merge(record_df, ats_df, on='UID')
gambling_df = pd.merge(df1, ou_df, on='UID')

# Drop repeat columns
columns_to_drop = ['Team_x', 'MOV_x', 'ATS_x', 'Year_x',
       'Team_y', 'MOV_y', 'ATS_y', 'Year_y']

gambling_df.drop(columns_to_drop, axis=1, inplace=True)

# Preview the Dataset
gambling_df.head(5)

,Win-Loss Record,Win %,UID,ATS Record,Cover %,Team,Over Record,Over %,Under %,Total,Year
0,17-2-0,89.5%,2003New England,14-5-0,73.7%,New England,9-10-0,47.4%,52.6%,-0.6,2003
1,13-4-0,76.5%,2003Kansas City,10-7-0,58.8%,Kansas City,11-6-0,64.7%,35.3%,5.5,2003
2,14-5-0,73.7%,2003Indianapolis,11-7-1,61.1%,Indianapolis,12-7-0,63.2%,36.8%,5.8,2003
3,13-5-0,72.2%,2003Philadelphia,11-7-0,61.1%,Philadelphia,9-8-1,52.9%,47.1%,0.9,2003
4,13-5-0,72.2%,2003Tennessee,10-8-0,55.6%,Tennessee,9-9-0,50.0%,50.0%,3.8,2003


In [4]:
# Reorder the columns
gambling_df = gambling_df[['Team', 'Year', 'UID', 
                           'Win-Loss Record', 'Win %', 
                           'ATS Record', 'Cover %', 
                           'Over Record', 'Over %', 'Under %', 'Total']]

In [5]:
# Investigate the dtypes
gambling_df.dtypes

Team                object
Year                 int64
UID                 object
Win-Loss Record     object
Win %               object
ATS Record          object
Cover %             object
Over Record         object
Over %              object
Under %             object
Total              float64
dtype: object

In [6]:
# Change year to datetime
gambling_df['Year'] = gambling_df['Year'].astype(str)

In [7]:
# Change the numbers to floating points
columns = ['Win %', 'Cover %', 'Over %', 'Under %']

for column in columns:
    gambling_df[column] = gambling_df[column].str.replace('%', '')
    gambling_df[column] = pd.to_numeric(gambling_df[column], errors='coerce')

In [8]:
# Split the record columns into individual columns and convert to integers
gambling_df[['Record Wins', 'Record Losses', 'Record Tie']] = gambling_df['Win-Loss Record'].str.split('-', expand=True)
gambling_df[['ATS Wins', 'ATS Losses', 'ATS Tie']] = gambling_df['ATS Record'].str.split('-', expand=True)
gambling_df[['OU Wins', 'OU Losses', 'OU Tie']] = gambling_df['Over Record'].str.split('-', expand=True)

# Convert all of these columns to integers
columns = ['Record Wins', 'Record Losses', 'Record Tie', 'ATS Wins', 'ATS Losses', 'ATS Tie', 'OU Wins', 'OU Losses', 'OU Tie']

for column in columns:
    gambling_df[column] = gambling_df[column].astype(int)

In [9]:
# Confirm the data cleaning operations were successful
gambling_df.dtypes

Team                object
Year                object
UID                 object
Win-Loss Record     object
Win %              float64
ATS Record          object
Cover %            float64
Over Record         object
Over %             float64
Under %            float64
Total              float64
Record Wins          int32
Record Losses        int32
Record Tie           int32
ATS Wins             int32
ATS Losses           int32
ATS Tie              int32
OU Wins              int32
OU Losses            int32
OU Tie               int32
dtype: object

In [11]:
# Preview the cleaned dataset
gambling_df.head(5)

,Team,Year,UID,Win-Loss Record,Win %,ATS Record,Cover %,Over Record,Over %,Under %,Total,Record Wins,Record Losses,Record Tie,ATS Wins,ATS Losses,ATS Tie,OU Wins,OU Losses,OU Tie
0,New England,2003,2003New England,17-2-0,89.5,14-5-0,73.7,9-10-0,47.4,52.6,-0.6,17,2,0,14,5,0,9,10,0
1,Kansas City,2003,2003Kansas City,13-4-0,76.5,10-7-0,58.8,11-6-0,64.7,35.3,5.5,13,4,0,10,7,0,11,6,0
2,Indianapolis,2003,2003Indianapolis,14-5-0,73.7,11-7-1,61.1,12-7-0,63.2,36.8,5.8,14,5,0,11,7,1,12,7,0
3,Philadelphia,2003,2003Philadelphia,13-5-0,72.2,11-7-0,61.1,9-8-1,52.9,47.1,0.9,13,5,0,11,7,0,9,8,1
4,Tennessee,2003,2003Tennessee,13-5-0,72.2,10-8-0,55.6,9-9-0,50.0,50.0,3.8,13,5,0,10,8,0,9,9,0


In [13]:
# Export the DataFrame to an .csv in the Resources directory
gambling_df.to_csv("resources/gambling.csv", index=False)

In [14]:
# Group the teams together
grouped_team = gambling_df.groupby('Team').agg({
    'Win %': 'mean', 'Cover %': 'mean', 'Over %': 'mean', 'Under %': 'mean', 'Total': 'mean', 
    'Record Wins': 'sum', 'Record Losses': 'sum', 'Record Tie': 'sum',
    'ATS Wins': 'sum', 'ATS Losses': 'sum', 'ATS Tie': 'sum',
    'OU Wins': 'sum', 'OU Losses': 'sum', 'OU Tie': 'sum',
}).round(2)

grouped_team
grouped_team.to_csv("resources/team_overall.csv")